# Dasherd EDA

polygon.io docs:  
https://polygon.io/docs/options/getting-started

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

from polygon import RESTClient

In [2]:
polygon_io_password = 'MDJr2zORlZBkgpCAlo_RpbjvbeFQmxJ1'

In [3]:
client = RESTClient(polygon_io_password)

In [101]:
client

In [5]:
stock_symbols = [
    'TSLA'
    , 'QQQ'
    , 'AAPL'
    , 'NVDA'
    , 'AMD'
    , 'AMZN'
    , 'MSFT'
    , 'NFLX'
    , 'META'
    , 'GOOGL'
    , 'T'
]
len(stock_symbols)

11

## Get stock prices for target dates

In [140]:
aggs = client.get_aggs(ticker="TSLA", 
                       multiplier=1, timespan="day", 
                       from_="2020-11-01", 
                       to="2022-07-03",
                       adjusted=None, sort=None, limit=None, params=None, raw=False)

# convert aggs to dataframe
aggs_df = pd.DataFrame(aggs)
aggs_df.shape

aggs_df['ticker'] = 'TSLA'
aggs_df.head()

,open,high,low,close,volume,vwap,timestamp,transactions,otc,ticker
0,394.00,406.9799,392.3000,400.51,29021118.0,398.6073,1604293200000,512507,None,TSLA
1,409.73,427.7700,406.6900,423.90,34333350.0,420.6331,1604379600000,607840,None,TSLA
2,430.62,435.4000,417.1000,420.98,32143057.0,423.7291,1604466000000,574859,None,TSLA
3,428.30,440.0000,424.0001,438.09,28414523.0,433.3403,1604552400000,526073,None,TSLA
4,436.10,436.5700,424.2800,429.95,21706014.0,429.6142,1604638800000,390722,None,TSLA


In [ ]:
# daily_prices_df = pd.DataFrame()
symbols_to_grab = [ticker for ticker in stock_symbols if ticker not in daily_prices_df['ticker'].unique()]
for ticker in symbols_to_grab[0:4]:
    aggs = client.get_aggs(ticker=ticker, 
                       multiplier=1, 
                       timespan="day", 
                       from_="2020-11-01", 
                       to="2022-07-03",
                       adjusted=None, sort=None, limit=None, params=None, raw=False)
    aggs_df = pd.DataFrame(aggs)
    aggs_df['ticker'] = ticker
    daily_prices_df = pd.concat([daily_prices_df, aggs_df])

In [ ]:
symbols_to_grab = [ticker for ticker in stock_symbols if ticker not in daily_prices_df['ticker'].unique()]
symbols_to_grab

#### Write out prices to csv locally

In [ ]:
pwd

In [ ]:
daily_prices_df.to_csv('daily_prices_backtest_22July.csv', index=False)

#### Read in stock prices from local csv

In [6]:
stock_prices_df = pd.read_csv('C:/Users/jerem/Desktop/dasherd/daily_prices_backtest_22July.csv')
stock_prices_df.shape

In [114]:
# Create analysis_df of only 11/19 stock prices
analysis_df = stock_prices_df.copy()
analysis_df['datetime'] = pd.to_datetime(analysis_df['timestamp']/1000,unit='s')
analysis_df = analysis_df[(1==1) & 
                          (analysis_df['datetime']==pd.Timestamp('2020-11-19 05:00:00'))]

analysis_df.shape

(10, 11)

In [139]:
analysis_df.head()

,open,high,low,close,volume,vwap,timestamp,transactions,otc,ticker,datetime
13,492.0000,508.6112,487.57,499.2700,62475346.0,499.7851,1605762000000,1129578,NaN,TSLA,2020-11-19 05:00:00
432,289.4800,292.7000,288.85,292.3800,22718422.0,291.2226,1605762000000,155948,NaN,QQQ,2020-11-19 05:00:00
851,117.5900,119.0600,116.81,118.6400,73687972.0,118.0320,1605762000000,492499,NaN,AAPL,2020-11-19 05:00:00
1270,132.1275,134.8750,131.00,134.4025,55493420.0,133.1654,1605762000000,273734,NaN,NVDA,2020-11-19 05:00:00
1689,82.3900,85.6400,81.72,85.5400,50116759.0,84.3769,1605762000000,339887,NaN,AMD,2020-11-19 05:00:00


In [110]:
# create strategy_opt_list - a list of dicts of options to be purchased
# Strategy A: 
    # Sell 1 call at stock price-20%, 
    # Buy 1 put at current price, 
    # Sell 1 put at stock price+20%, 
    # and Buy 2 calls at current stock price
strategy_opt_list = [
    {
        'volume': 1,
        'action':'sell',
        'option_type':'call',
        'price_diff_pct': -0.20
    },
    {
        'volume': 1,
        'action':'buy',
        'option_type':'put',
        'price_diff_pct': 0
    },
    {
        'volume': 1,
        'action':'sell',
        'option_type':'put',
        'price_diff_pct': 0.20
    },
    {
        'volume': 2,
        'action':'buy',
        'option_type':'call',
        'price_diff_pct': 0
    },
]

In [111]:
strategy_opt_list[0]

{'volume': 1, 'action': 'sell', 'option_type': 'call', 'price_diff_pct': -0.2}

In [112]:
def create_option_string(ticker, exp_date, option_type, stock_price, price_diff_pct):
    """
    Function to create option string
    
    exp_date: YYMMDD format
    option_type: {call|put}
    option_price_string: 
    
    """
    if option_type == 'call':
        call_put_val = 'C'
    else:
        call_put_val = 'P'
    
    option_price = 5 * round(stock_price * (1 + price_diff_pct)/5)
    option_price_string = '00' + str(option_price) + '000'

    option_ticker_string = 'O:' + ticker + expiration_date + call_put_val + option_price_string
    return option_ticker_string

In [113]:
create_option_string(ticker='TSLA', 
                     exp_date='210219', 
                     option_type='call', 
                     stock_price=499.27, 
                     price_diff_pct=-0.20)

'O:TSLA210219C00400000'

In [170]:
def calc_option_profit(option_ticker_string: str, 
                       opt_prices_from_date, 
                       opt_prices_to_date, 
                       opt_start_price_date,
                       opt_end_price_date,
                       buy_sell: str,
                       opt_volume: int):
    """
    Function to calculate option profit
    """
    print(f'Getting daily option prices for {option_ticker_string}')

    print(f'Daily option prices being gathered from {opt_prices_from_date} to {opt_prices_to_date}', end=',')

    aggs = client.get_aggs(ticker=option_ticker_string, 
                       multiplier=1, 
                       timespan="day", 
                       from_=opt_prices_from_date, 
                       to=opt_prices_to_date,
                       adjusted=None, sort=None, limit=None, params=None, raw=False)
    opts_df = pd.DataFrame(aggs)
    print(f'Daily prices fetched: {opts_df.shape[0]}')
    opts_df['datetime'] = pd.to_datetime(opts_df['timestamp']/1000,unit='s')
    opt_start_price = opts_df[opts_df['datetime']==opt_start_price_date]['close'].iloc[0]
    print(f'Option start price: {opt_start_price}')
    opt_end_price = opts_df[opts_df['datetime']==opt_end_price_date]['close'].iloc[0]
    print(f'Option end price: {opt_end_price}')

    if buy_sell == 'sell':
        option_profit = opt_volume * (opt_start_price - opt_end_price)
    else:
        option_profit = opt_volume * (opt_end_price - opt_start_price)        
    print(f'Option profit: {option_profit}')
    return option_profit

In [138]:
calc_option_profit(option_ticker_string='O:TSLA210219C00400000', 
                   opt_prices_from_date = "2020-11-01",
                   opt_prices_to_date = "2021-02-19",
                   opt_start_price_date = pd.Timestamp('2020-11-19 05:00:00'),
                   opt_end_price_date = pd.Timestamp('2021-02-19 05:00:00'),
                   buy_sell = 'buy',
                   opt_volume=1)

Getting daily option prices for O:TSLA210219C00400000
Daily option prices being gathered from 2020-11-01 to 2021-02-19
Daily prices fetched: 71
Option start price: 128.2
Option end price: 380.0
Option profit: 251.8


251.8

In [ ]:
# pseudocode for full experiment for one strategy
    # experiment defines purchase/sell date, expiration date
    # strategy defines what options to purchase/sell

strategy_profit = 0
for ticker in stock_symbols:
    ticker_profit = 0
    for option_config in strategy_dict:
        # create option tickers, grab option prices, calculate profit
        ticker_profit += option_profit
    strategy_profit += option_profit

## Define Strategy

In [124]:
# create strategy_opt_list - a list of dicts of options to be purchased
# Strategy A: 
    # Sell 1 call at stock price-20%, 
    # Buy 1 put at current price, 
    # Sell 1 put at stock price+20%, 
    # and Buy 2 calls at current stock price

strategy_opt_list = [
    {
        'volume': 1,
        'action':'sell',
        'option_type':'call',
        'price_diff_pct': -0.20
    },
    {
        'volume': 1,
        'action':'buy',
        'option_type':'put',
        'price_diff_pct': 0
    },
    {
        'volume': 1,
        'action':'sell',
        'option_type':'put',
        'price_diff_pct': 0.20
    },
    {
        'volume': 2,
        'action':'buy',
        'option_type':'call',
        'price_diff_pct': 0
    },
]

print(strategy_opt_list[0])

In [143]:
stock_symbols

['TSLA',
 'QQQ',
 'AAPL',
 'NVDA',
 'AMD',
 'AMZN',
 'MSFT',
 'NFLX',
 'META',
 'GOOGL',
 'T']

In [172]:
stock_symbol_dict = {ticker: {'profit': None, 
                              'start_price':None,
                              'options_missing_price':0
                             } 
                     for ticker in stock_symbols}
stock_symbol_dict

{'TSLA': {'profit': None, 'start_price': None, 'options_missing_price': 0},
 'QQQ': {'profit': None, 'start_price': None, 'options_missing_price': 0},
 'AAPL': {'profit': None, 'start_price': None, 'options_missing_price': 0},
 'NVDA': {'profit': None, 'start_price': None, 'options_missing_price': 0},
 'AMD': {'profit': None, 'start_price': None, 'options_missing_price': 0},
 'AMZN': {'profit': None, 'start_price': None, 'options_missing_price': 0},
 'MSFT': {'profit': None, 'start_price': None, 'options_missing_price': 0},
 'NFLX': {'profit': None, 'start_price': None, 'options_missing_price': 0},
 'META': {'profit': None, 'start_price': None, 'options_missing_price': 0},
 'GOOGL': {'profit': None, 'start_price': None, 'options_missing_price': 0},
 'T': {'profit': None, 'start_price': None, 'options_missing_price': 0}}

In [173]:
# create dataframe from stock symbol results dict
# results_df = pd.DataFrame.from_dict(stock_symbol_dict, orient='index')
# results_df['profit'].sum()
# results_df.head()

In [174]:
for ticker in stock_symbols[1:3]:
    print(ticker)

QQQ
AAPL


In [175]:
# Requires stock_symbols list of tickers to evaluate
# Requires analysis_df with close price of all evaluated tickers on purchase/sell date of experiment

# Strategy A
# Backtesting experiment: All options purchased/sold on 11/19/2020, expire 90 days later on 2/19/2021
    # TODO: update this to grab price on 2/17 (the Friday prior to the expiration)

# TODO: clean up this date shit
option_string_expiration_date = '210219'
opt_prices_from_date = "2020-11-01"
opt_prices_to_date = "2021-02-19"
opt_start_price_dt = pd.Timestamp('2020-11-19 05:00:00')
opt_end_price_dt = pd.Timestamp('2021-02-19 05:00:00')

stock_symbol_dict = {ticker: {'profit': None, 
                              'start_price':None,
                              'options_missing_price':0
                             } 
                     for ticker in stock_symbols}

#for ticker in stock_symbols:  # run full loop
for ticker in stock_symbols[1:3]: # run loop for first few tickers
#for ticker in ['TSLA']:  # run loop for one example ticker
    print(f'Evaluating options for ticker: {ticker}')
    ticker_profit = 0
    # TODO: update this price lookup to be at option issue date, instead of requiring analysis_df to only include prices at option issue date
    # TODO: add check to make sure ticker price at date is in prices
    ticker_price_at_purchase = analysis_df[analysis_df['ticker']==ticker]['close'].iloc[0]
    stock_symbol_dict[ticker]['start_price'] = ticker_price_at_purchase
    for option_config in strategy_opt_list:
        print(f'Option config: {option_config}')
        loop_option_string = create_option_string(ticker=ticker, 
                                                  exp_date=option_string_expiration_date, 
                                                  option_type=option_config['option_type'], 
                                                  stock_price=ticker_price_at_purchase, 
                                                  price_diff_pct=option_config['price_diff_pct']
                                                 )
        print('Calculating option profit')
        try:
            option_profit = calc_option_profit(option_ticker_string=loop_option_string,
                                               opt_prices_from_date = opt_prices_from_date,
                                               opt_prices_to_date = opt_prices_to_date,
                                               opt_start_price_date = opt_start_price_dt,
                                               opt_end_price_date = opt_end_price_dt,
                                               buy_sell = option_config['action'],
                                               opt_volume=option_config['volume'])
            ticker_profit += option_profit
        except:
            print('Option price does not exist, skipping')
            stock_symbol_dict[ticker]['options_missing_price'] += 1
            
        time.sleep(20)  # rest after each option, API is limited to like 6 calls / minute

    stock_symbol_dict[ticker]['profit'] = ticker_profit
    print(f'Profit for ticker {ticker}: {ticker_profit}')

results_df = pd.DataFrame.from_dict(stock_symbol_dict, orient='index')
strategy_profit = results_df['profit'].sum()
print(f'Overall strategy profit: {strategy_profit}')

print(results_df)

Evaluating options for ticker: QQQ
Option config: {'volume': 1, 'action': 'sell', 'option_type': 'call', 'price_diff_pct': -0.2}
Calculating option profit
Getting daily option prices for O:QQQ210219C00235000
Daily option prices being gathered from 2020-11-01 to 2021-02-19,Daily prices fetched: 13
Option price does not exist, skipping
Option config: {'volume': 1, 'action': 'buy', 'option_type': 'put', 'price_diff_pct': 0}
Calculating option profit
Getting daily option prices for O:QQQ210219P00290000
Daily option prices being gathered from 2020-11-01 to 2021-02-19,Daily prices fetched: 75
Option start price: 13.93
Option end price: 0.01
Option profit: -13.92
Option config: {'volume': 1, 'action': 'sell', 'option_type': 'put', 'price_diff_pct': 0.2}
Calculating option profit
Getting daily option prices for O:QQQ210219P00350000
Daily option prices being gathered from 2020-11-01 to 2021-02-19,Daily prices fetched: 46
Option price does not exist, skipping
Option config: {'volume': 2, 'action

In [ ]:
# create proper experiment class

class OptionsExperiment:
    """Options Experiment Class"""
    def __init__(self, purchase_date, expiration_date):
        self.purchase_date = purchase_date
        self.expiration_date = expiration_date


experiment = OptionsExperiment("2020-11-19", "2021-02-19")

print(experiment.purchase_date)
print(experiment.expiration_date)

In [21]:
analysis_df['stock_price_put'] = 5 * round(analysis_df['close']/5)
analysis_df['stock_price_call'] = 5 * round(analysis_df['close']/5)
analysis_df['stock_price_plus_20pct_put'] = analysis_df['close'] * 1.2
analysis_df['stock_price_plus_20pct_put'] = 5 * round(analysis_df['stock_price_plus_20pct_put']/5)  # rounded to nearest 5
analysis_df['stock_price_minus_20pct_call'] = analysis_df['close'] * 0.8
analysis_df['stock_price_minus_20pct_call'] = 5 * round(analysis_df['stock_price_minus_20pct_call']/5)  # rounded to nearest 5
analysis_df["sell_date90"] = analysis_df["datetime"] + timedelta(days=90)
analysis_df.head()

## Get options data

In [ ]:
"""
A January 21, 2022 Call Option for Uber with a $50 Strike Price

UBER220121C00050000 = UBER + 220121 + C + 00050000

Underlying Stock - UBER
Expiration Date - January 21st, 2022 or ‘220121’ (YYMMDD)
Option Type - Call or ‘C’
Strike Price - 00050000 (50000/1000) or $50
"""

In [ ]:
# TSLA call option purchased on 11/2/2020 for stock price minus 20% of 320
#  expiration date of 90 days in future of purchase - 1/31/2021
# TSLA 210131 C 00320000
# TSLA 210219 C 00400000

# 2 put calls at strike price, 1 put at 20%, 1 call at 20%

In [24]:
aggs = client.get_aggs(ticker="O:TSLA210219C00400000", 
                       multiplier=1, 
                       timespan="day", 
                       from_="2020-11-01", 
                       to="2021-02-19",
                       adjusted=None, sort=None, limit=None, params=None, raw=False)

In [25]:
opts_df = pd.DataFrame(aggs)
opts_df.shape

(71, 9)

In [26]:
opts_df['datetime'] = pd.to_datetime(opts_df['timestamp']/1000,unit='s')

In [117]:
opts_df.tail()

,open,high,low,close,volume,vwap,timestamp,transactions,otc,datetime
66,393.75,401.23,389.15,401.23,13,394.6354,1613106000000,6,None,2021-02-12 05:00:00
67,407.80,407.80,395.28,395.28,67,405.8333,1613451600000,4,None,2021-02-16 05:00:00
68,377.55,394.37,371.55,393.81,20,379.6305,1613538000000,8,None,2021-02-17 05:00:00
69,381.50,390.29,381.50,389.50,7,386.1643,1613624400000,5,None,2021-02-18 05:00:00
70,391.83,392.10,380.00,380.00,108,383.4605,1613710800000,15,None,2021-02-19 05:00:00


### Other options methods to explore

In [ ]:
opt_contracts = client.get_options_contract(
    # ticker="O:TSLA210129C00700000", 
    # ticker="TSLA210131C00320000", 
    ticker="TSLA", 
    as_of='2021-01-31',
    params = None, raw = False,
    )

In [ ]:
options = []
for t in client.list_options_contracts(
        underlying_ticker = 'TSLA',
        #underlying_ticker_lt: Optional[str] = None,
        #underlying_ticker_lte: Optional[str] = None,
        #underlying_ticker_gt: Optional[str] = None,
        #underlying_ticker_gte: Optional[str] = None,
        contract_type = 'call',
        expiration_date = '2021-01-29',
        # expiration_date_lt = '2021-01-01',
        # expiration_date_gt = '2021-01-29',
        #expiration_date_lt: Optional[Union[str, date]] = None,
        #expiration_date_lte: Optional[Union[str, date]] = None,
        #expiration_date_gt: Optional[Union[str, date]] = None,
        #expiration_date_gte: Optional[Union[str, date]] = None,
        as_of = '2021-01-02',
        # strike_price = 320,
        strike_price_lt = 820,
        strike_price_gt = 420,
        #strike_price_lt: Optional[float] = None,
        #strike_price_lte: Optional[float] = None,
        #strike_price_gt: Optional[float] = None,
        #strike_price_gte: Optional[float] = None,
        #expired: Optional[bool] = None,
        #limit: Optional[int] = None,
        #sort: Optional[Union[str, Sort]] = None,
        #order: Optional[Union[str, Order]] = None,
        #params: Optional[Dict[str, Any]] = None,
        #raw = False,
    ):
    options.append(t)
print(options)